In [3]:

library(numbat)
library(dplyr)
library(Seurat)
library(ggplot2)
library(glue)
library(data.table)
library(ggtree)
library(stringr)
library(tidygraph)
library(patchwork)


# Provide sample name as argument
pat<- commandArgs()[6]

mypal = c('1' = 'gray', '2' = "#377EB8", '3' = "#4DAF4A", '4' = "#984EA3", 
          '5'="#ff9768", '6'='#ae1717', '7'='#0f04b5', '8'="#f87382",
          '9'='green','10'='yellow','11'='deeppink')

# Read-in numbat output
nb = Numbat$new(out_dir = paste0('708/'))

# Sync-in sample Seurat object from AWS
seu<- readRDS(paste0("data_Sarcoma708GEX_genes_300_UMI_600_annotated_for_infercnv.rds"))
seu$barcode_orig <- rownames(seu@meta.data)

# Single-cell CNV calls
cnv_calls<- nb$joint_post %>% select(cell, CHROM, seg, cnv_state, p_cnv, p_cnv_x, p_cnv_y)
table(cnv_calls$cnv_state)
cnv_calls %>% group_by(cnv_state) %>% arrange(p_cnv,desc=F)

# Clone info
clones<-dim(table(nb$clone_post$clone_opt))
clone_info<-nb$clone_post
seu$cell<-seu$barcode_orig
seu@meta.data<-left_join(seu@meta.data,clone_info,by='cell')
rownames(seu@meta.data)<-seu$barcode_orig

### Plots
pdf(paste0("708/jana_numbat_plots.pdf"),width = 10)
# Copy number landscape and single-cell phylogeny
nb$plot_phylo_heatmap(clone_bar = TRUE, p_min = 0.9,raster = T,pal_clone = mypal)

# Consensus copy number segments
nb$plot_consensus()

# Bulk CNV profiles
nb$bulk_clones %>% 
  filter(n_cells > 50) %>%
  plot_bulks(min_LLR = 10, # filtering CNVs by evidence
             legend = TRUE,raster=T)

# clones
DimPlot(seu, group.by = 'clone_opt',shuffle = T, raster=T,cols = mypal[1:clones])
DimPlot(seu, group.by = 'GT_opt',shuffle = T, raster=T)

# Tumor versus normal probability
p1<-FeaturePlot(seu, features  = c('p_cnv'),order = T, raster=T)+
  scale_color_gradient2(low = 'royalblue', mid = 'white', high = 'red3', midpoint = 0.5, limits = c(0,1), name = 'Posterior')+
  ggtitle('Tumor vs normal probability\n(joint)')

p2<-FeaturePlot(seu, features  = c('p_cnv_x'),order = T, raster=T)+
  scale_color_gradient2(low = 'royalblue', mid = 'white', high = 'red3', midpoint = 0.5, limits = c(0,1), name = 'Posterior')+
  ggtitle('Tumor vs normal probability\n(gex)')

p3<-FeaturePlot(seu, features  = c('p_cnv_y'),order = T, raster=T)+
  scale_color_gradient2(low = 'royalblue', mid = 'white', high = 'red3', midpoint = 0.5, limits = c(0,1), name = 'Posterior')+
  ggtitle('Tumor vs normal probability\n(allele)')
print((p1+p2 +p3)+plot_layout(ncol=2))

# Tumor phylogeny
nb$plot_sc_tree(
  label_size = 3, 
  branch_width = 0.5, 
  tip_length = 0.5,
  pal_clone = mypal,
  tip = TRUE)

# mutational history
nb$plot_mut_history(pal=mypal)
dev.off()


   amp   bdel    del    loh 
257649  24538 269918 282187 

cell,CHROM,seg,cnv_state,p_cnv,p_cnv_x,p_cnv_y
<chr>,<fct>,<chr>,<chr>,<dbl>,<dbl>,<dbl>
TCAGCTCCACACCGAC-1,10,10e,del,1.834043e-29,2.988076e-01,4.303828e-29
ATCTGCCAGATGCCTT-1,10,10e,del,5.778790e-29,2.988076e-01,1.356071e-28
AGATTGCAGCGATATA-1,3,3e,loh,8.528403e-29,4.994979e-01,8.545547e-29
GTTCGGGCAAAGGCGT-1,12,12b,del,4.430667e-28,9.982990e-01,1.424324e-17
TCAGCTCCACACCGAC-1,3,3b,loh,5.062524e-25,3.756470e-01,8.414288e-25
AACTCAGGTTCGTTGA-1,10,10e,del,3.412318e-24,2.988076e-01,8.007463e-24
CCTTTCTCAACACCTA-1,12,12b,del,3.695716e-24,3.571977e-01,4.965300e-17
CTGCCTAGTGAACCTT-1,3,3e,loh,1.199307e-23,4.994979e-01,1.201718e-23
AGGGATGTCGCATGAT-1,10,10e,del,1.363569e-23,2.988076e-01,3.199798e-23


Warning message:
“The `x` argument of `as_tibble.matrix()` must have unique column names if `.name_repair` is omitted as of tibble 2.0.0.
ℹ Using compatibility `.name_repair`.
ℹ The deprecated feature was likely used in the treeio package.
  Please report the issue at <https://github.com/YuLab-SMU/treeio/issues>.”
Warning message:
“Removed 2 rows containing missing values (`geom_tile()`).”
Warning message:
“ggrepel: 4 unlabeled data points (too many overlaps). Consider increasing max.overlaps”
Warning message:
“ggrepel: 9 unlabeled data points (too many overlaps). Consider increasing max.overlaps”
Warning message:
“ggrepel: 3 unlabeled data points (too many overlaps). Consider increasing max.overlaps”
Warning message:
“ggrepel: 9 unlabeled data points (too many overlaps). Consider increasing max.overlaps”
Warning message:
“ggrepel: 11 unlabeled data points (too many overlaps). Consider increasing max.overlaps”
Warning message:
“ggrepel: 4 unlabeled data points (too many overlaps). Cons

png 
  2